In [1]:
!pip install Census
!pip install us

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import us
from census import Census
from us import states
#from census_data import 

# Census API Key
from config import api_key
api_key
c = Census(api_key, year=2013)
c1 = Census(api_key,year=2017)

In [2]:
male_data=c1.acs1.get('B24010_019E', {'for': 'state:*'})
m_pd = pd.DataFrame(male_data)
m_pd = m_pd.rename(columns={"B24010_019E": "Male", 
                                       "state": "State Code"})
m_pd = m_pd[["State Code","Male"]]
m_pd.head()

,State Code,Male
0,01,72636.0
1,02,13748.0
2,04,130196.0
3,05,33942.0
4,06,922373.0


In [3]:
female_data=c1.acs1.get('B24010_155E', {'for': 'state:*'})
f_pd = pd.DataFrame(female_data)
f_pd = f_pd.rename(columns={"B24010_155E": "Female", 
                                       "state": "State Code"})
f_pd = f_pd[["State Code","Female"]]
f_pd.head()

,State Code,Female
0,01,123251.0
1,02,21991.0
2,04,221393.0
3,05,72013.0
4,06,1348070.0


In [4]:
# get state names
state_dict = us.states.mapping('fips', 'name')
# Append DataFrame
mf_df = pd.merge(f_pd, m_pd, on='State Code', how='inner')
mf_df=mf_df.fillna(0)

mf_df["Total"] = mf_df["Male"].astype(int) + mf_df["Female"].astype(int)
mf_df["State"] = ''
mf_df["Type"] = ''
#mf_df["State"] = state_dict[mf_df["State Code"]] 
state_dict = us.states.mapping('fips', 'name')
for index, row in mf_df.iterrows():
    mf_df.loc[index,'State'] = state_dict[mf_df.loc[index,'State Code']]
    mf_df.loc[index,'Type'] = 'Manufacturing'


mf_df = mf_df[["State Code", "State", "Type","Male", "Female","Total"]]
mf_df.sort_values('Total', ascending=False).head()

,State Code,State,Type,Male,Female,Total
4,06,California,Manufacturing,922373.0,1348070.0,2270443
43,48,Texas,Manufacturing,573538.0,894007.0,1467545
32,36,New York,Manufacturing,331486.0,673544.0,1005030
9,12,Florida,Manufacturing,299000.0,629087.0,928087
13,17,Illinois,Manufacturing,252640.0,442071.0,694711


In [5]:
#used functions
def func_merge_df(df1, df2, column):
    mf_df = pd.merge(df1, df2, on=column, how='inner')
    return mf_df

def func_getdata(key1, key2, jobtype):
    male_data= []
    female_data = []
    state_dict = us.states.mapping('fips', 'name')
    
    male_data=c1.acs1.get(key1, {'for': 'state:*'})
    female_data=c1.acs1.get(key2, {'for': 'state:*'})
    m_pd = pd.DataFrame(male_data)
    f_pd = pd.DataFrame(female_data)
    # Merge the dataframe
    mf_df = func_merge_df(df1=f_pd, df2=m_pd, column='state')
    mf_df=mf_df.fillna(0)
    # Get the total 
    mf_df["State Name"] = ''
    mf_df["Type"] = ''
    # Get the State name
    for index, row in mf_df.iterrows():
        mf_df.loc[index,'State Name'] = state_dict[mf_df.loc[index,'state']]
        mf_df.loc[index,'Type'] = jobtype
    return mf_df

In [6]:
#main program
cnt = 1
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_155E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 

# First time for the header
    if cnt == 1:
        rs_data.to_csv("job_census_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1

    if cnt > 1:
        with open('job_census_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed


In [7]:
cnt = 1
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_155E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)
 
 
key1_val = 'B24040_007E'
key2_val = 'B24040_111E'
jobtypeVal = 'Manufacturing'

rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
rs_data
rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
rs_data.sort_values('Total', ascending=False).head() 

,State Code,State Name,Type,Male,Female,Total
4,06,California,Manufacturing,1020705.0,421201.0,1441906
43,48,Texas,Manufacturing,730346.0,242316.0,972662
22,26,Michigan,Manufacturing,570745.0,191694.0,762439
35,39,Ohio,Manufacturing,543443.0,187144.0,730587
13,17,Illinois,Manufacturing,463010.0,187851.0,650861
